In [2]:
import pandas as pd
file_path = '/Users/yujinlee/Downloads/'
df = pd.read_csv(file_path + 'combined_training_before_normalize.txt', sep='\t')
print(df.shape)

/var/folders/yw/mp5v5fdx1_n4b01714b8bdl80000gn/T/ipykernel_67419/3878672483.py:2: DtypeWarning: Columns (12,15,16,18,21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path + 'combined_training.txt', sep='\t')


(6695102, 29)


In [17]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

features = df[['region', 'city', 'adExchange', 'width', 'height', 'floorPrice', 'weekday', 'hour']]
target = df['numClick']

numerical_cols = ['region', 'city', 'width', 'height', 'floorPrice', 'hour','adExchange']
categorical_cols = ['weekday']  # Assuming 'adExchange' needs encoding if non-numeric

In [25]:
features.tail()

,region,city,adExchange,width,height,floorPrice,weekday,hour
6695097,216,222,3.0,980,90,20,Monday,16
6695098,55,56,2.0,200,200,4,Monday,11
6695099,216,227,3.0,180,600,20,Monday,11
6695100,253,254,2.0,336,280,5,Monday,12
6695101,146,159,3.0,1000,90,20,Monday,17


In [40]:
# Creating transformers for numerical and categorical data
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combining transformers into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Applying the transformations
X_preprocessed = preprocessor.fit_transform(features)
X_preprocessed.shape  # Display the shape of the transformed feature matrix to confirm completion


(6695102, 14)

In [42]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Split the data into training and testing sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, target, test_size=0.3, random_state=42)

# Initialize the Logistic Regression model
logistic_model = LogisticRegression()

# Train the model
logistic_model.fit(X_train, y_train)

# Predict on the testing set
y_pred = logistic_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Predict on the testing set
y_pred = logistic_model.predict(X_test)
print("Coefficients:", logistic_model.coef_)
print("Intercept:", logistic_model.intercept_)


/Users/yujinlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yujinlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Coefficients: [[ 0.32184193 -0.33938767  0.00865819  0.01192459  0.16994901  0.01982248
  -0.15417052 -0.19533879 -0.24058476 -0.01714379  0.89247962 -0.17634483
  -0.08402347 -0.19860977]]
Intercept: [-7.1269368]


/Users/yujinlee/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [46]:
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", report)

Accuracy: 0.9992706112078927
Confusion Matrix:
 [[2007066       0]
 [   1465       0]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00   2007066
           1       0.00      0.00      0.00      1465

    accuracy                           1.00   2008531
   macro avg       0.50      0.50      0.50   2008531
weighted avg       1.00      1.00      1.00   2008531



In [44]:
# Access the feature names from the one-hot encoder and numerical features
feature_names = (numerical_cols +
                 list(preprocessor.named_transformers_['cat'].named_steps['onehot']
                      .get_feature_names_out(categorical_cols)))

# Get the coefficients from the logistic regression model
coefficients = logistic_model.coef_[0]

# Create a DataFrame to view the feature names alongside their corresponding coefficients
import pandas as pd
feature_importance = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})

# Sort the features by the absolute values of their coefficients for better visibility
feature_importance.sort_values(by='Coefficient', key=abs, ascending=False, inplace=True)

print(feature_importance)


              Feature  Coefficient
10     weekday_Sunday     0.892480
1                city    -0.339388
0              region     0.321842
8      weekday_Monday    -0.240585
13  weekday_Wednesday    -0.198610
7      weekday_Friday    -0.195339
11   weekday_Thursday    -0.176345
4          floorPrice     0.169949
6          adExchange    -0.154171
12    weekday_Tuesday    -0.084023
5                hour     0.019822
9    weekday_Saturday    -0.017144
3              height     0.011925
2               width     0.008658
